In [31]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Activation, Add, MaxPooling2D, BatchNormalization, Dropout , Input, Dense, Conv2D, MaxPool2D , AveragePooling2D, ReLU, LeakyReLU, Flatten, GlobalAveragePooling2D

from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.models import Model
from keras.regularizers import l2
from tensorflow.keras import initializers
from keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
(x_train, y_train), (x_test, y_test) = np.load('/content/drive/MyDrive/Colab_Notebooks/brain/brain_aug.npy', allow_pickle=True)


In [5]:
x_train.shape

(2200, 224, 224, 3)

In [6]:
# normalize
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255


In [7]:
# one Hot
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


In [8]:
y_test[0]

array([0., 1.], dtype=float32)

In [9]:
x_test.max()

1.0

In [17]:
# creat_model

i = Input(shape=(224,224,3))

x = Conv2D(32, 3, (1,1), 'same', kernel_regularizer=regularizers.L2(0.0001),kernel_initializer= initializers.RandomNormal())(i)
x = Activation('elu')(x)
x = BatchNormalization()(x)
x = Conv2D(64, 3, (1,1), 'same', kernel_regularizer=regularizers.L2(0.0001),kernel_initializer= initializers.RandomNormal())(x)
x = Activation('elu')(x)
x = AveragePooling2D(strides=2)(x)
x = BatchNormalization()(x)

x = Dropout(0.2)(x)

x = Conv2D(64, 3, (1,1), 'same', kernel_regularizer=regularizers.L2(0.0001),kernel_initializer= initializers.RandomNormal())(x)
x = Activation('elu')(x)
x = AveragePooling2D(strides=2)(x)
x = BatchNormalization()(x)
x = Conv2D(64, 3, (2,2), 'same', kernel_regularizer=regularizers.L2(0.0001),kernel_initializer= initializers.RandomNormal())(x)
x = Activation('elu')(x)
x = AveragePooling2D(strides=2)(x)
x = BatchNormalization()(x)

x = Dropout(0.3)(x)

x = Conv2D(128, 3, (1,1), 'same', kernel_regularizer=regularizers.L2(0.0001),kernel_initializer= initializers.RandomNormal())(x)
x = Activation('elu')(x)
x = AveragePooling2D(strides=2)(x)
x = BatchNormalization()(x)

x = Dropout(0.4)(x)

x = GlobalAveragePooling2D()(x)

x = Dense(units=2, activation='softmax')(x)

model = Model(i, x)

model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_8 (Conv2D)           (None, 224, 224, 32)      896       
                                                                 
 activation_8 (Activation)   (None, 224, 224, 32)      0         
                                                                 
 batch_normalization_8 (Batc  (None, 224, 224, 32)     128       
 hNormalization)                                                 
                                                                 
 conv2d_9 (Conv2D)           (None, 224, 224, 64)      18496     
                                                                 
 activation_9 (Activation)   (None, 224, 224, 64)      0         
                                                           

In [18]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=12)
checkpoint_filepath = '/content/drive/MyDrive/brain/best_w'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

opt = tf.keras.optimizers.Adam(lr = 0.001, beta_1=0.9)

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [19]:
model.fit(x_train, y_train, batch_size=64, epochs=50, validation_split=0.2, callbacks= [callback, model_checkpoint_callback])

Epoch 1/50
31/31 [==============================] - 793s 26s/step - loss: 0.6198 - accuracy: 0.6899 - val_loss: 0.9850 - val_accuracy: 0.5818
Epoch 2/50
31/31 [==============================] - 811s 26s/step - loss: 0.5462 - accuracy: 0.7283 - val_loss: 1.2720 - val_accuracy: 0.5818
Epoch 3/50
31/31 [==============================] - 818s 26s/step - loss: 0.5303 - accuracy: 0.7505 - val_loss: 1.3706 - val_accuracy: 0.5818
Epoch 4/50
31/31 [==============================] - 801s 26s/step - loss: 0.5203 - accuracy: 0.7571 - val_loss: 0.9550 - val_accuracy: 0.5818
Epoch 5/50
31/31 [==============================] - 822s 27s/step - loss: 0.5079 - accuracy: 0.7657 - val_loss: 1.1810 - val_accuracy: 0.5909
Epoch 6/50
31/31 [==============================] - 815s 26s/step - loss: 0.5187 - accuracy: 0.7606 - val_loss: 1.5430 - val_accuracy: 0.5864
Epoch 7/50
31/31 [==============================] - 833s 27s/step - loss: 0.4955 - accuracy: 0.7742 - val_loss: 0.6430 - val_accuracy: 0.6818
Epoch 

In [42]:
model.load_weights(checkpoint_filepath)
opt = tf.keras.optimizers.Adam(lr = 0.0001, beta_1=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.evaluate(x_test, y_test)
model.fit(x_train, y_train, batch_size=64, epochs=50, validation_split=0.2, callbacks= [callback, model_checkpoint_callback])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


19/19 [==============================] - 2s 96ms/step - loss: 0.3202 - accuracy: 0.8919
Epoch 1/50
28/28 [==============================] - 17s 574ms/step - loss: 0.3578 - accuracy: 0.8767 - val_loss: 0.4952 - val_accuracy: 0.7864
Epoch 2/50
28/28 [==============================] - 16s 568ms/step - loss: 0.3503 - accuracy: 0.8898 - val_loss: 0.5755 - val_accuracy: 0.7386
Epoch 3/50
28/28 [==============================] - 16s 569ms/step - loss: 0.3366 - accuracy: 0.8881 - val_loss: 0.5943 - val_accuracy: 0.7341
Epoch 4/50
28/28 [==============================] - 16s 571ms/step - loss: 0.3378 - accuracy: 0.8943 - val_loss: 0.4570 - val_accuracy: 0.8023
Epoch 5/50
28/28 [==============================] - 16s 572ms/step - loss: 0.3369 - accuracy: 0.8864 - val_loss: 0.4412 - val_accuracy: 0.8159
Epoch 6/50
28/28 [==============================] - 16s 571ms/step - loss: 0.3444 - accuracy: 0.8761 - val_loss: 0.4330 - val_accuracy: 0.8295
Epoch 7/50
28/28 [==============================] - 16

In [48]:
model.load_weights(checkpoint_filepath)
opt = tf.keras.optimizers.Adam(lr = 0.0001, beta_1=0.9)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

OpError: ignored

In [49]:
model.evaluate(x_test, y_test)


19/19 [==============================] - 2s 92ms/step - loss: 0.6988 - accuracy: 0.7393


[0.698801577091217, 0.739279568195343]

In [46]:
opt = tf.keras.optimizers.Adam(lr = 0.0001, beta_1=0.9)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.evaluate(x_test, y_test)
model.fit(x_train, y_train, batch_size=64, epochs=50, validation_split=0.2, callbacks= [callback, model_checkpoint_callback])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


19/19 [==============================] - 2s 96ms/step - loss: 0.3561 - accuracy: 0.8576
Epoch 1/50
28/28 [==============================] - 17s 533ms/step - loss: 0.3636 - accuracy: 0.8619 - val_loss: 0.7378 - val_accuracy: 0.7568
Epoch 2/50
28/28 [==============================] - 15s 524ms/step - loss: 0.3482 - accuracy: 0.8744 - val_loss: 0.9787 - val_accuracy: 0.6682
Epoch 3/50
28/28 [==============================] - 15s 525ms/step - loss: 0.3540 - accuracy: 0.8790 - val_loss: 0.4909 - val_accuracy: 0.8205
Epoch 4/50
28/28 [==============================] - 15s 527ms/step - loss: 0.3201 - accuracy: 0.8847 - val_loss: 0.5140 - val_accuracy: 0.7818
Epoch 5/50
28/28 [==============================] - 15s 528ms/step - loss: 0.3222 - accuracy: 0.8864 - val_loss: 0.5483 - val_accuracy: 0.7750
Epoch 6/50
28/28 [==============================] - 15s 527ms/step - loss: 0.3156 - accuracy: 0.8926 - val_loss: 1.1748 - val_accuracy: 0.5227
Epoch 7/50
28/28 [==============================] - 15

KeyboardInterrupt: ignored